## Constants in BN256 

repo: https://github.com/privacy-scaling-explorations/halo2curves/tree/main/src/bn256 , https://github.com/Consensys/gnark-crypto/tree/v0.12.0/ecc/bn254 \
paper: https://eprint.iacr.org/2010/354.pdf

Given an integer $ 0 \leq n < 2^{256}$, return its 4 limbs in little-endian where each 64-bit limb is in big endian.

In [128]:
def u_64_little_endian(n):
    str_hex = hex(n)
    str_hex_without_0x = str_hex[2:]
    full_width_str = '0' * (64 - len(str_hex_without_0x)) + str_hex_without_0x
    assert len(full_width_str) == 64

    res = []
    for i in range(4):
        temp = '0x' + full_width_str[64 - 16 * (i + 1) : 64 - 16 * i]
        res.append(temp)
    return res

### The Base Field $\mathbb{F}_q$
seed $U = 4965661367192848881$ \
the base field modulus $q = 36 U^4 + 36 U^3 + 24 U^2 + 6 U + 1$

In [129]:
U = 4965661367192848881
q_int = 36 * U**4 + 36 * U**3 + 24 * U**2 + 6 * U + 1
hex(q_int)

'0x30644e72e131a029b85045b68181585d97816a916871ca8d3c208c16d87cfd47'

The limbs of $q$ are referred to: https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/bn256/fq.rs#L34-L40 

In [130]:
u_64_little_endian(q_int)  

['0x3c208c16d87cfd47',
 '0x97816a916871ca8d',
 '0xb85045b68181585d',
 '0x30644e72e131a029']

In [131]:
Fq = GF(q_int)
Fq

Finite Field of size 21888242871839275222246405745257275088696311157297823662689037894645226208583

### The Scalar Field $\mathbb{F}_r$
the scalar field modulus: $r = 36 U^4 + 36 U^3 + 18 U^2 + 6 U+1$

In [132]:
r_int = 36 * U**4 + 36 * U**3 + 18 * U**2 + 6 * U + 1
hex(r_int)

'0x30644e72e131a029b85045b68181585d2833e84879b9709143e1f593f0000001'

The limbs of $r$ are referred to: https://github.com/privacy-scaling-explorations/halo2curves/blob/main/src/bn256/fr.rs#L49-L55

In [133]:
u_64_little_endian(r_int)

['0x43e1f593f0000001',
 '0x2833e84879b97091',
 '0xb85045b68181585d',
 '0x30644e72e131a029']

In [134]:
Fr = GF(r_int)
Fr

Finite Field of size 21888242871839275222246405745257275088548364400416034343698204186575808495617

### The equation of BN256:
$$E / \mathbb{F}_{q}: y^2 = x^3 + 3$$

In [135]:
E = EllipticCurve(Fq, [0, 3])
E

Elliptic Curve defined by y^2 = x^3 + 3 over Finite Field of size 21888242871839275222246405745257275088696311157297823662689037894645226208583

The order of the group $E(\mathbb{F}_{q})$ is $r$ which is prime, thus the group is cyclic and $E(\mathbb{F}_{q}) = \mathbb{G}_{1}$

In [136]:
E.cardinality() == r_int

True

In [137]:
E.abelian_group()

Additive abelian group isomorphic to Z/21888242871839275222246405745257275088548364400416034343698204186575808495617 embedded in Abelian group of points on Elliptic Curve defined by y^2 = x^3 + 3 over Finite Field of size 21888242871839275222246405745257275088696311157297823662689037894645226208583

We take the generator of $\mathbb{G}_{1}$ as $(1, 2)$

In [138]:
G1_GENERATOR_X = Fq(1)
G1_GENERATOR_Y = Fq(2)
# the constructor of elliptic curve point
G1_GENERATOR = E(G1_GENERATOR_X, G1_GENERATOR_Y)
G1_GENERATOR.order() == r_int

True

$\zeta_{q}$ is one of the two $3rd$ primitive roots of unity in $\mathbb{F}_{q}$, the other one is $\zeta_{q}^{2}$.

In [139]:
zetaq = Fq.zeta(3)
zetaq

21888242871839275220042445260109153167277707414472061641714758635765020556616

$\zeta_{r}$ is one of the two $3rd$ primitive roots of unity in $\mathbb{F}_{r}$, the other one is $\zeta_{r}^{2}$.


In [140]:
zetar = Fr.zeta(3)
zetar

4407920970296243842393367215006156084916469457145843978461

The endomorphism of $E/\mathbb{F}_{q}$ defined by:
$$(x, y) \mapsto (\zeta x, y)$$
where $\zeta$ is a primitive $3rd$ root

In [141]:
def endo (E, zeta, P):
    (xp, yp) = P.xy()
    return E(zeta * xp, yp)
    

We know 
$$(\zeta_{q} x, y) = [\zeta_{r}] (x, y)$$
but we should find the right $\zeta_{q}$ which maybe $\zeta_{q}$ or $\zeta_{q}^2$, the same applies to $\zeta_{r}$

In [142]:
zP = endo(E, zetaq, G1_GENERATOR)
if zP == int(zetar)*G1_GENERATOR:
    print("1st trial")
else:
    zetar = zetar^2
    if zP == int(zetar)*G1_GENERATOR: 
        print("2nd trial")
    else: 
        zetaq = zetaq^2
        zP = endo(E, zetaq, G1_GENERATOR)
        if zP == int(zetar)*G1_GENERATOR:
            print("3nd trial")
        else:
            zetar = zetar^2
            assert zP == int(zetar)*G1_GENERATOR
            print("4th trial")
(zetaq, zetar)

2nd trial


(21888242871839275220042445260109153167277707414472061641714758635765020556616,
 21888242871839275217838484774961031246154997185409878258781734729429964517155)

The bit lengths of $\zeta_{q}$ and $\zeta_{r}$ are 

In [143]:
(Integer(zetaq).bit_length(), Integer(zetar).bit_length())

(254, 254)

### The GLV method
The references are:
https://hackmd.io/@drouyang/glv and 
https://www.iacr.org/archive/crypto2001/21390189.pdf

Given two integers $n$ and $\lambda$, GLV method outputs two short vectors $v_1 = (a_1, b_1), v_2 = (a_2, b_2)$ such that 
$$ a_1 + b_1 \lambda = 0\mod n $$
$$ a_2 + b_2 \lambda = 0\mod n $$

In [144]:
def ext_euclidean(a, b):
    assert a <= b
    sqr_b = sqrt(b)
    u = a
    v = b
    x1 = 1
    y1 = 0
    x2 = 0
    y2 = 1

    X = []
    Y = []
    R = []

    flag = -1
    i = -1
    while not u == 0:
        q = v // u
        r = v - q * u
        if flag == -1:
            if r < sqr_b:
                flag = i # exactly the largest index i such that R[i] >= sqrt_b
        i = i + 1
        R.append(r)  # R[i] = r
        x = x2 - q * x1
        X.append(x)
        y = y2 - q * y1  # ax + by = d
        Y.append(y)
        v = u
        u = r
        x2 = x1
        x1 = x
        y2 = y1
        y1 = y
    
    d = v
    x = x2
    y = y2
    X.append(x)
    Y.append(y)
    R.append(0)

    # ((r_l, t_l), (r_l_1, t_l_1), (r_l_2, t_l_2))
    return ((R[flag], X[flag]), (R[flag + 1], X[flag + 1]), (R[flag + 2], X[flag + 2]))  

def GLV(scalar, n):
    ((r_l, t_l), (r_l_1, t_l_1), (r_l_2, t_l_2)) = ext_euclidean(scalar, n)
    b_1 = t_l_1  # to be consistent with the repo, we indeed take -b_1 here
    a_1 = r_l_1
    if (r_l^2 + t_l^2) <= (r_l_2^2 + t_l_2^2):
        b_2 = -t_l
        a_2 = r_l
    else:
        b_2 = -t_l_2
        a_2 = r_l_2

    return ((a_1, b_1), (a_2, b_2))  # thus the two short vectors: v_1 = (a_1, -b_1) and v_2 = (a_2, b_2)
    

In [145]:
scalar = int(zetar)
n = r_int

The values $b_1, b_2$ can be referred to: https://github.com/privacy-scaling-explorations/halo2curves/blob/6abfdbb0eefd476864e1ed91d32ab84c7b36c203/src/bn256/curve.rs#L131-L132

In [146]:
((a1, b1), (a2, b2)) = GLV(scalar, n)
(u_64_little_endian(b1), u_64_little_endian(b2))

(['0x89d3256894d213e3',
  '0x0000000000000000',
  '0x0000000000000000',
  '0x0000000000000000'],
 ['0x0be4e1541221250b',
  '0x6f4d8248eeb859fd',
  '0x0000000000000000',
  '0x0000000000000000'])

We take $\gamma_1 = \lfloor -b_1 \cdot 2^{256} / r \rceil$, as $ -b_1 / r < 1$. Recall the variable $b_1$ in the code represents $-b_1$ for $b_1$ is the coordinate of $v_1$. \
And $\gamma_2 = \lfloor b_2 \cdot 2^{256} / r \rceil$, where $b_2$ is the coordinate of $v_2$. \
Such values can be referred to: https://github.com/privacy-scaling-explorations/halo2curves/blob/6abfdbb0eefd476864e1ed91d32ab84c7b36c203/src/bn256/curve.rs#L127-L130 \
(the comment in the repo is not correct)

In [147]:
gamma1 = round(b1 * 2^256 / r_int ) 
u_64_little_endian(gamma1)

['0xd91d232ec7e0b3d7',
 '0x0000000000000002',
 '0x0000000000000000',
 '0x0000000000000000']

In [148]:
gamma2 = round(b2 * 2^256 / r_int )
u_64_little_endian(gamma2)

['0x5398fd0300ff6565',
 '0x4ccef014a773d2d2',
 '0x0000000000000002',
 '0x0000000000000000']

Consequently, given an arbitrary scalar $k$, we use GLV method to decompose: \
$k = k_1 + k_2 \times \zeta$
where $k_1, k_2$ are almost the half bit length of $k$ (recall $k$ with $256$ bits)

To make consistent with Algo 3.74 in https://hackmd.io/@drouyang/glv, we have to clarify the code above each line in the notations used in Algo 3.74.

In [149]:
def decompose(k):
    c1_ = gamma2 * k
    c2_ = gamma1 * k
    # round(b2 * k / n)
    c1 = c1_ >> 256
    # round(-b1 * k / n)
    c2 = c2_ >> 256
    # c1 * -b1
    q1 = c1 * b1
    # c2 * b2
    q2 = c2 * b2
    # -c1 * b1 - c2 * b2
    k2 = q1 - q2
    # k - c1 * a1 - c2 * a2
    k1 = k - c1 * a1 - c2 * a2
    # the correctness of GLV involves the following constraints about bit lengths and identity
    return (k1.bit_length()<=128 and k2.bit_length() <=128 and Fr(k) == Fr(k1 + k2 * int(zetar)))
    

Unit test for 1000 random scalar values:

In [150]:
for i in range(1000):
    k = int(Fr.random_element())
    if decompose(k) == False:
        print("the algorithm is broken")
        break
print("pass")

pass
